# Baltimore Analytics — Feature Engineering
**Project:** `baltimore-analytics`  
**Input:** `analytics` dataset  
**Output:** `analytics.neighborhood_features` (feature matrix)  
**Author:** Spencer  

---
### What This Notebook Does
Aggregates all spatially enriched tables to the **neighborhood (NSA) level** and builds a single
feature matrix suitable for clustering and Looker Studio dashboards.

```
analytics.crime_incidents_legacy    ─┐
analytics.crime_incidents_current   ─┤
analytics.bpd_arrests               ─┤
analytics.vacant_building_notices   ─┤→  analytics.neighborhood_features
analytics.vacant_building_rehabs    ─┤
analytics.building_permits          ─┤
analytics.service_requests_311      ─┤
analytics.real_property             ─┘
```

### Feature Groups
| Group | Features |
|---|---|
| **Demographics** | Population, age distribution, race/ethnicity, household composition |
| **Public Safety** | Crime rate, arrest rate, crime trend (YoY) |
| **Housing** | Vacancy rate, rehab activity, building permit investment |
| **Quality of Life** | 311 request volume, request types |
| **Property** | Median assessed value, owner vs renter ratio |

---

## 0. Configuration

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s — %(levelname)s — %(message)s')
log = logging.getLogger(__name__)

GCP_PROJECT       = "baltimore-analytics"
ANALYTICS_DATASET = "analytics"
GCP_REGION        = "us-east1"
CREDENTIALS_PATH  = "service_account.json"

# Reference years for trend analysis
CURRENT_YEAR  = 2024
BASELINE_YEAR = 2019  # Pre-COVID baseline

print(f"✓ Config loaded.")

## 1. Initialize Client

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    CREDENTIALS_PATH,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
bq = bigquery.Client(project=GCP_PROJECT, credentials=credentials)
log.info(f"✓ BigQuery client ready.")

## 2. Pre-Flight Check
Verify all `analytics` tables exist before building features.

In [ ]:
REQUIRED_TABLES = [
    "crime_incidents_legacy",
    "crime_incidents_current",
    "bpd_arrests",
    "vacant_building_notices",
    "vacant_building_rehabs",
    "building_permits",
    "service_requests_311",
    "real_property",
    "neighborhood_boundaries",
]

all_ok = True
for table in REQUIRED_TABLES:
    try:
        t = bq.get_table(f"{GCP_PROJECT}.{ANALYTICS_DATASET}.{table}")
        print(f"✅ {table}: {t.num_rows:,} rows")
    except Exception as e:
        print(f"❌ {table}: missing")
        all_ok = False

print(f"\n{'✓ All tables present.' if all_ok else '✗ Fix missing tables before proceeding.'}")

## 3. Helper — Run and Write Query
Utility to run a SQL query and write result to `analytics` dataset.

In [ ]:
def run_and_write(sql: str, dest_table: str, description: str = "") -> int:
    """Run a SQL query and write result to analytics dataset. Returns row count."""
    dest = f"{GCP_PROJECT}.{ANALYTICS_DATASET}.{dest_table}"
    job_config = bigquery.QueryJobConfig(
        destination=dest,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        create_disposition=bigquery.CreateDisposition.CREATE_IF_NEEDED,
    )
    log.info(f"Building: {dest_table}{' — ' + description if description else ''}")
    bq.query(sql, job_config=job_config).result()
    rows = bq.get_table(dest).num_rows
    log.info(f"  ✓ {rows:,} rows → {dest}")
    return rows

print("✓ Helper defined.")

## 4. Build Intermediate Feature Tables
Each section aggregates one source table to neighborhood level.  
Results are written to intermediate tables in `analytics`, then joined in Cell 5.

In [ ]:
# ── 4.1 CRIME FEATURES ────────────────────────────────────────────────────
# Combines legacy (pre-2021) and current (post-2021) into unified crime counts
run_and_write(f"""
    WITH combined AS (
        SELECT neighborhood, crimedatetime, description
        FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.crime_incidents_legacy`
        WHERE neighborhood IS NOT NULL
        UNION ALL
        SELECT neighborhood, crimedatetime, description
        FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.crime_incidents_current`
        WHERE neighborhood IS NOT NULL
    )
    SELECT
        neighborhood,
        COUNT(*) AS crime_total,
        COUNTIF(EXTRACT(YEAR FROM crimedatetime) = {CURRENT_YEAR})  AS crime_{CURRENT_YEAR},
        COUNTIF(EXTRACT(YEAR FROM crimedatetime) = {BASELINE_YEAR}) AS crime_{BASELINE_YEAR},
        COUNTIF(EXTRACT(YEAR FROM crimedatetime) >= 2021)            AS crime_post2021,
        COUNTIF(EXTRACT(YEAR FROM crimedatetime) <  2021)            AS crime_pre2021,
        -- Top crime type
        APPROX_TOP_COUNT(description, 1)[OFFSET(0)].value           AS top_crime_type,
        -- YoY trend: positive = increasing crime
        SAFE_DIVIDE(
            COUNTIF(EXTRACT(YEAR FROM crimedatetime) = {CURRENT_YEAR}) -
            COUNTIF(EXTRACT(YEAR FROM crimedatetime) = {CURRENT_YEAR} - 1),
            NULLIF(COUNTIF(EXTRACT(YEAR FROM crimedatetime) = {CURRENT_YEAR} - 1), 0)
        ) AS crime_yoy_change
    FROM combined
    GROUP BY neighborhood
""", "_feat_crime", "crime aggregation")

In [ ]:
# ── 4.2 ARREST FEATURES ───────────────────────────────────────────────────
run_and_write(f"""
    SELECT
        neighborhood,
        COUNT(*) AS arrest_total,
        COUNTIF(EXTRACT(YEAR FROM arrestdatetime) = {CURRENT_YEAR})  AS arrest_{CURRENT_YEAR},
        COUNTIF(EXTRACT(YEAR FROM arrestdatetime) = {BASELINE_YEAR}) AS arrest_{BASELINE_YEAR},
        APPROX_TOP_COUNT(chargedescription, 1)[OFFSET(0)].value      AS top_arrest_charge
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.bpd_arrests`
    WHERE neighborhood IS NOT NULL
    GROUP BY neighborhood
""", "_feat_arrests", "arrest aggregation")

In [ ]:
# ── 4.3 VACANCY FEATURES ──────────────────────────────────────────────────
run_and_write(f"""
    SELECT
        n.neighborhood,
        COUNT(DISTINCT v.objectid)                                     AS vacant_notice_count,
        COUNT(DISTINCT r.objectid)                                     AS rehab_permit_count,
        SAFE_DIVIDE(COUNT(DISTINCT r.objectid), COUNT(DISTINCT v.objectid)) AS rehab_to_vacant_ratio
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.vacant_building_notices` v
    FULL OUTER JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}.vacant_building_rehabs` r
        USING (neighborhood)
    RIGHT JOIN (SELECT DISTINCT neighborhood FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_boundaries`) n
        USING (neighborhood)
    WHERE n.neighborhood IS NOT NULL
    GROUP BY n.neighborhood
""", "_feat_vacancy", "vacancy aggregation")

In [ ]:
# ── 4.4 BUILDING PERMIT FEATURES ──────────────────────────────────────────
run_and_write(f"""
    SELECT
        neighborhood,
        COUNT(*)                                                        AS permit_total,
        COUNTIF(EXTRACT(YEAR FROM issueddate) >= 2020)                  AS permit_recent_5yr,
        COUNTIF(EXTRACT(YEAR FROM issueddate) = {CURRENT_YEAR})         AS permit_{CURRENT_YEAR},
        SUM(CAST(estimatedcost AS FLOAT64))                             AS permit_total_value,
        AVG(CAST(estimatedcost AS FLOAT64))                             AS permit_avg_value
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.building_permits`
    WHERE neighborhood IS NOT NULL
    GROUP BY neighborhood
""", "_feat_permits", "building permit aggregation")

In [ ]:
# ── 4.5 311 SERVICE REQUEST FEATURES ──────────────────────────────────────
run_and_write(f"""
    SELECT
        neighborhood,
        COUNT(*)                                                                AS requests_311_total,
        COUNTIF(EXTRACT(YEAR FROM createddate) = {CURRENT_YEAR})                AS requests_311_{CURRENT_YEAR},
        COUNTIF(EXTRACT(YEAR FROM createddate) = {BASELINE_YEAR})               AS requests_311_{BASELINE_YEAR},
        APPROX_TOP_COUNT(servicerequesttype, 1)[OFFSET(0)].value                AS top_311_request_type,
        COUNTIF(LOWER(servicerequesttype) LIKE '%abandon%'
             OR LOWER(servicerequesttype) LIKE '%vacant%')                      AS requests_311_blight
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.service_requests_311`
    WHERE neighborhood IS NOT NULL
    GROUP BY neighborhood
""", "_feat_311", "311 aggregation")

In [ ]:
# ── 4.6 REAL PROPERTY FEATURES ────────────────────────────────────────────
# Note: real_property stores parcel polygons (not points) — we derive centroids
# at query time using ST_CENTROID(ST_GEOGFROMTEXT(..., make_valid => TRUE))
# to handle duplicate vertices in some parcel geometries.
# Assessed value = currland + currimpr (fullcash is 0 for 93% of parcels)
run_and_write(f"""
    SELECT
        n.name                                                                              AS neighborhood,
        COUNT(*)                                                                            AS parcel_count,
        APPROX_QUANTILES(CAST(p.currland AS FLOAT64) + CAST(p.currimpr AS FLOAT64), 2)[OFFSET(1)] AS median_assessed_value,
        AVG(CAST(p.currland AS FLOAT64) + CAST(p.currimpr AS FLOAT64))                    AS avg_assessed_value,
        SUM(CAST(p.currland AS FLOAT64) + CAST(p.currimpr AS FLOAT64))                    AS total_assessed_value,
        COUNTIF(LOWER(p.usegroup) LIKE '%residential%')                                   AS residential_parcels,
        COUNTIF(LOWER(p.usegroup) LIKE '%commercial%')                                    AS commercial_parcels
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.real_property` p
    JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_boundaries` n
      ON ST_WITHIN(
           ST_CENTROID(ST_GEOGFROMTEXT(p.geo_polygon_wkt, make_valid => TRUE)),
           n.geo_polygon_wkt
         )
    WHERE p.geo_polygon_wkt IS NOT NULL
    GROUP BY n.name
""", "_feat_property", "real property aggregation")


## 5. Build Master Feature Matrix
Join all intermediate feature tables to `neighborhood_boundaries` as the spine.  
Every neighborhood gets a row — features are NULL if no data exists for that neighborhood.

In [ ]:
run_and_write(f"""
    SELECT
        -- ── IDENTITY ──────────────────────────────────────────────────────
        nb.name                             AS neighborhood,
        nb.objectid                         AS nsa_objectid,

        -- ── DEMOGRAPHICS (from neighborhood_boundaries) ───────────────────
        nb.population,
        nb.white,
        nb.blk_afam,
        nb.hisp_lat,
        nb.asian,
        nb.med_age,
        nb.total_units,
        nb.occ_vacant,
        SAFE_DIVIDE(nb.occ_vacant, nb.total_units)   AS housing_vacancy_rate,
        SAFE_DIVIDE(nb.tenure_owner, nb.hh_total)    AS owner_occupancy_rate,
        SAFE_DIVIDE(nb.tenure_renter, nb.hh_total)   AS renter_occupancy_rate,

        -- ── CRIME ─────────────────────────────────────────────────────────
        COALESCE(cr.crime_total, 0)                  AS crime_total,
        COALESCE(cr.crime_{CURRENT_YEAR}, 0)         AS crime_{CURRENT_YEAR},
        COALESCE(cr.crime_{BASELINE_YEAR}, 0)        AS crime_{BASELINE_YEAR},
        cr.crime_yoy_change,
        cr.top_crime_type,
        SAFE_DIVIDE(cr.crime_total, nb.population)   AS crime_per_capita,

        -- ── ARRESTS ───────────────────────────────────────────────────────
        COALESCE(ar.arrest_total, 0)                 AS arrest_total,
        COALESCE(ar.arrest_{CURRENT_YEAR}, 0)        AS arrest_{CURRENT_YEAR},
        ar.top_arrest_charge,
        SAFE_DIVIDE(ar.arrest_total, nb.population)  AS arrest_per_capita,

        -- ── VACANCY ───────────────────────────────────────────────────────
        COALESCE(va.vacant_notice_count, 0)          AS vacant_notice_count,
        COALESCE(va.rehab_permit_count, 0)           AS rehab_permit_count,
        va.rehab_to_vacant_ratio,
        SAFE_DIVIDE(va.vacant_notice_count, nb.total_units) AS vacant_notices_per_unit,

        -- ── BUILDING PERMITS ──────────────────────────────────────────────
        COALESCE(bp.permit_total, 0)                 AS permit_total,
        COALESCE(bp.permit_recent_5yr, 0)            AS permit_recent_5yr,
        bp.permit_total_value,
        bp.permit_avg_value,
        SAFE_DIVIDE(bp.permit_total_value, nb.population) AS permit_value_per_capita,

        -- ── 311 ───────────────────────────────────────────────────────────
        COALESCE(sr.requests_311_total, 0)           AS requests_311_total,
        COALESCE(sr.requests_311_{CURRENT_YEAR}, 0)  AS requests_311_{CURRENT_YEAR},
        sr.top_311_request_type,
        COALESCE(sr.requests_311_blight, 0)          AS requests_311_blight,
        SAFE_DIVIDE(sr.requests_311_total, nb.population) AS requests_311_per_capita,

        -- ── PROPERTY VALUES ───────────────────────────────────────────────
        pr.parcel_count,
        pr.median_assessed_value,
        pr.avg_assessed_value,
        pr.total_assessed_value,
        pr.residential_parcels,
        pr.commercial_parcels,

        -- ── METADATA ──────────────────────────────────────────────────────
        '{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}' AS _feature_built_at

    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_boundaries` nb
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_crime`    cr ON nb.name = cr.neighborhood
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_arrests`  ar ON nb.name = ar.neighborhood
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_vacancy`  va ON nb.name = va.neighborhood
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_permits`  bp ON nb.name = bp.neighborhood
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_311`      sr ON nb.name = sr.neighborhood
    LEFT JOIN `{GCP_PROJECT}.{ANALYTICS_DATASET}._feat_property` pr ON nb.name = pr.neighborhood
""", "neighborhood_features", "master feature matrix")

print("\n✓ neighborhood_features table built.")

## 6. Validate Feature Matrix

In [ ]:
# Row count — should be 279 (one per neighborhood)
t = bq.get_table(f"{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features")
print(f"Rows: {t.num_rows} (expected 279)")
print(f"Columns: {len(t.schema)}")
print()

# Null rates per feature — flag anything unexpectedly sparse
cols = [f.name for f in t.schema if f.field_type in ('FLOAT64', 'INT64', 'FLOAT', 'INTEGER')]
null_checks = ",\n".join([f"ROUND(COUNTIF({c} IS NULL) / COUNT(*) * 100, 1) AS {c}_null_pct" for c in cols[:20]])

result = bq.query(f"""
    SELECT {null_checks}
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features`
""").to_dataframe()

null_summary = result.T.rename(columns={0: 'null_pct'}).sort_values('null_pct', ascending=False)
print("Null rates for numeric features:")
display(null_summary[null_summary['null_pct'] > 0])

In [ ]:
# Sample top 10 neighborhoods by crime per capita
print("Top 10 neighborhoods by crime per capita:")
display(bq.query(f"""
    SELECT
        neighborhood,
        population,
        crime_total,
        ROUND(crime_per_capita, 4)      AS crime_per_capita,
        vacant_notice_count,
        ROUND(median_assessed_value, 0) AS median_assessed_value
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features`
    WHERE population > 100  -- exclude parks/industrial areas
    ORDER BY crime_per_capita DESC
    LIMIT 10
""").to_dataframe())

# Sample top 10 neighborhoods by investment (permit value per capita)
print("\nTop 10 neighborhoods by permit investment per capita:")
display(bq.query(f"""
    SELECT
        neighborhood,
        population,
        permit_total,
        ROUND(permit_value_per_capita, 0) AS permit_value_per_capita,
        ROUND(median_assessed_value, 0)   AS median_assessed_value
    FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features`
    WHERE population > 100
    ORDER BY permit_value_per_capita DESC
    LIMIT 10
""").to_dataframe())  

## 7. Export Feature Matrix for Clustering
Pull `neighborhood_features` into a pandas DataFrame for use in notebook 04.  
Saves a local CSV as a checkpoint.

In [ ]:
features_df = bq.query(f"""
    SELECT * FROM `{GCP_PROJECT}.{ANALYTICS_DATASET}.neighborhood_features`
    ORDER BY neighborhood
""").to_dataframe()

features_df.to_csv("neighborhood_features.csv", index=False)

print(f"✓ Exported {len(features_df)} rows × {len(features_df.columns)} columns")
print(f"  Saved to: neighborhood_features.csv")
print(f"\nNumeric features available for clustering:")
numeric_cols = features_df.select_dtypes(include='number').columns.tolist()
for col in numeric_cols:
    print(f"  {col}")

---
## Next Steps

1. **`04_clustering.ipynb`** — K-means segmentation using `neighborhood_features`
2. **`05_looker_studio.md`** — Connect `analytics.neighborhood_features` to Looker Studio

**Notes on feature quality:**
- `bpd_arrests`: ~43% of records missing geo → arrest features undercount for older records
- `service_requests_311`: 2023-2026 excluded from source data
- `real_property`: `assessedvalue` may be NULL for some parcels — coerced to FLOAT64
- Neighborhoods with `population < 100` are likely parks/industrial areas — consider excluding from clustering

**If intermediate tables need to be rebuilt individually:**
```python
# Re-run just one intermediate table
# Then re-run Cell 5 to rebuild the master matrix
```